In [2]:
import json
import awswrangler as wr
import boto3
import datetime as dt
import os

# Función que agrega columnas de año, mes y día a partir de una columna de fecha.
def add_partition_fields(df, formato = '%Y', col = "anio"):
    df['year'] = [x for x in df[col]]
    return df

#def lambda_handler(event, context):

dynamoDBTable = "cen-ingesta-calidad-dato-dev-config"#os.environ['dynamoDBTable'] 
element = "costo_marginal_proyectado"#os.environ['element'] 
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table(dynamoDBTable)

#Lectura del elemento en la tabla de Dynamo:
config = table.get_item(
        Key={
            'CEN': '{}'.format(element)
        }
    ).get('Item')
data = config ['params'] #Dentro de "params" se encuentran todos los parámetros necesarios para trabajar con la tabla antes listada.
data_params = json.loads(data)

pet_origen = data_params.get('pet_origen')
S3_origin = pet_origen + "year_partition=" + str(dt.datetime.now().year - 2) + "/"
pet_destino = data_params.get('pet_destino')
table = data_params.get('table')
write_mode = data_params.get('write_mode')
database_analytics = data_params.get('database_analytics')

df = wr.s3.read_parquet(path = S3_origin)
df["fecha"] = "01" + "-" + df["mes"].astype(str) + "-" + df["year"].astype(str)
#### Cambiando nombre a columnas 
columns = {'hidro':'hidrologia',
                'year':'anio',
                'barnom':'nombre_barra'}

df.rename(columns = columns, inplace = True)
df = add_partition_fields(df)
df["yearParticion"] = str(dt.datetime.now().year - 2)
print(df.dtypes)
wr.s3.to_parquet(
    df=df, path=pet_destino, index=False,
    dataset=True,compression = 'gzip',
    mode = write_mode,database=database_analytics,
    table=table,
    partition_cols = ["yearParticion","year"]
)
    

hidrologia        string
anio               Int64
mes                Int64
nombre_barra      string
cmg_horas        float64
duracion           Int64
cmg_mensual      float64
escenario         string
fecha             object
year               int64
yearParticion     object
dtype: object


{'paths': ['s3://cen-ingesta.calidad-analytics-dev/planificacion-y-desarrollo/fact_cmg_proyectado/yearparticion=2021/year=2021/3035badf01e440038cdbcb7bda63fbda.gz.parquet',
  's3://cen-ingesta.calidad-analytics-dev/planificacion-y-desarrollo/fact_cmg_proyectado/yearparticion=2021/year=2022/3035badf01e440038cdbcb7bda63fbda.gz.parquet',
  's3://cen-ingesta.calidad-analytics-dev/planificacion-y-desarrollo/fact_cmg_proyectado/yearparticion=2021/year=2023/3035badf01e440038cdbcb7bda63fbda.gz.parquet',
  's3://cen-ingesta.calidad-analytics-dev/planificacion-y-desarrollo/fact_cmg_proyectado/yearparticion=2021/year=2024/3035badf01e440038cdbcb7bda63fbda.gz.parquet',
  's3://cen-ingesta.calidad-analytics-dev/planificacion-y-desarrollo/fact_cmg_proyectado/yearparticion=2021/year=2025/3035badf01e440038cdbcb7bda63fbda.gz.parquet',
  's3://cen-ingesta.calidad-analytics-dev/planificacion-y-desarrollo/fact_cmg_proyectado/yearparticion=2021/year=2026/3035badf01e440038cdbcb7bda63fbda.gz.parquet',
  's3:/